In [1]:
'''
Pipeline issue to be resolved...
Exact same code here but it works
Run every week if needed

'''
import os
import pandas as pd
import glob
from datetime import datetime
from bs4 import BeautifulSoup
from utils.marketinsights import get_city_mapping, get_contact_information, get_industry, get_intersection, get_phone_mapping, label_country_by_city, label_country_by_phone, final_country, convert_to_datetime, AutoModel, AutoTokenizer

model_name = 'nomic-ai/nomic-embed-text-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
file_pattern = f"temp_{selection}_data_*.csv"
files = glob.glob(os.path.join(directory, file_pattern))
print(files)
latest_file = max(files, key=os.path.getctime)
print(latest_file)

try:
    existing = pd.read_csv('utils/data/Scrape/phoneextensions.csv')
except FileNotFoundError:
    existing = pd.DataFrame()

phone_storage = get_phone_mapping(existing)
city_storage = get_city_mapping()

df = pd.read_csv(latest_file)
df.head()

/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<All keys matched successfully>


['./utils/data/Scraped News/temp_marketinsights_data_2024-08-13.csv']
./utils/data/Scraped News/temp_marketinsights_data_2024-08-13.csv


,title,link,ticker,date,source,Article content,raw,raw2,People
0,Certain A Shares of CanSino Biologics Inc. are...,https://www.marketscreener.com/quote/stock/CAN...,6185,2024-08-12,S&P Capital IQ,Certain A Shares of CanSino Biologics Inc. are...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Xue Feng Yu', 'Age': '60 year', 'Po..."
1,German defence firm Renk appoints Manz-Siebje ...,https://www.marketscreener.com/quote/stock/REN...,R3NK,Aug. 12,Reuters,Aug 12 (Reuters) - German defence firm Renk Gr...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Susanne Wiegand', 'Age': '-', 'Posi..."
2,IRobot Corporation Appoints Michael J. Loparco...,https://www.marketscreener.com/quote/stock/IRO...,IRBT,Aug. 12,S&P Capital IQ,iRobot Corp. announced the addition of Michael...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Gary Cohen', 'Age': '63 year', 'Pos..."
3,"Charter Communications, Inc. Appoints Simon Ca...",https://www.marketscreener.com/quote/stock/CHA...,CHTR,Aug. 12,S&P Capital IQ,"Charter Communications, Inc. announced that Si...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Christopher Winfrey', 'Age': '48 ye..."
4,NaaS Technology Inc. Announces Chief Financial...,https://www.marketscreener.com/quote/stock/NAA...,REDU,Aug. 12,S&P Capital IQ,NaaS Technology Inc. announced that Mr. Alex W...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Yang Wang', 'Age': '36 year', 'Posi..."


In [2]:
df['Industry'] = df['raw'].apply(get_industry)
df['Contact Information'] = df['raw'].apply(get_contact_information)
df['Country_phone'] = df['Contact Information'].apply(lambda x: label_country_by_phone(x, phone_storage))
df['Country_city'] = df['Contact Information'].apply(lambda x: label_country_by_city(x, city_storage))
df['Country_candidates'] = df.apply(lambda x: get_intersection(x.Country_phone, x.Country_city), axis=1)
df['Country'] = df.apply(lambda x: final_country(x['Contact Information'], x.Country_candidates, tokenizer, model), axis=1)

df.head()

/Users/launchpad/workspace/services/Argus/utils/marketinsights.py:187: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 187 of the file /Users/launchpad/workspace/services/Argus/utils/marketinsights.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(soup)
/Users/launchpad/workspace/services/Argus/utils/marketinsights.py:208: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser

,title,link,ticker,date,source,Article content,raw,raw2,People,Industry,Contact Information,Country_phone,Country_city,Country_candidates,Country
0,Certain A Shares of CanSino Biologics Inc. are...,https://www.marketscreener.com/quote/stock/CAN...,6185,2024-08-12,S&P Capital IQ,Certain A Shares of CanSino Biologics Inc. are...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Xue Feng Yu', 'Age': '60 year', 'Po...",Healthcare,"{'Company Name': 'CanSino Biologics Inc.', 'Ad...",{China},{China},{China},China
1,German defence firm Renk appoints Manz-Siebje ...,https://www.marketscreener.com/quote/stock/REN...,R3NK,Aug. 12,Reuters,Aug 12 (Reuters) - German defence firm Renk Gr...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Susanne Wiegand', 'Age': '-', 'Posi...",Consumer Cyclicals,"{'Company Name': 'RENK Group AG', 'Address Lin...",{Germany},{Germany},{Germany},Germany
2,IRobot Corporation Appoints Michael J. Loparco...,https://www.marketscreener.com/quote/stock/IRO...,IRBT,Aug. 12,S&P Capital IQ,iRobot Corp. announced the addition of Michael...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Gary Cohen', 'Age': '63 year', 'Pos...",Technology,"{'Company Name': 'iRobot Corporation', 'Addres...","{United States, Canada}","{United States, United Kingdom}",{United States},United States
3,"Charter Communications, Inc. Appoints Simon Ca...",https://www.marketscreener.com/quote/stock/CHA...,CHTR,Aug. 12,S&P Capital IQ,"Charter Communications, Inc. announced that Si...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Christopher Winfrey', 'Age': '48 ye...",Technology,"{'Company Name': 'Charter Communications, Inc....","{United States, Canada}","{United States, United Kingdom}",{United States},United States
4,NaaS Technology Inc. Announces Chief Financial...,https://www.marketscreener.com/quote/stock/NAA...,REDU,Aug. 12,S&P Capital IQ,NaaS Technology Inc. announced that Mr. Alex W...,"<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","<!DOCTYPE html>\n<html lang=""en-US"" prefix=""og...","[{'Name': 'Yang Wang', 'Age': '36 year', 'Posi...",Utilities,{},{Unknown},{Unknown},{Unknown},Unknown


In [3]:
# Apply the conversion functions
df['Time'] = df['date'].apply(convert_to_datetime)
df.drop(['date', 'raw', 'raw2'], axis=1, inplace=True)
current_date = datetime.now().strftime("%Y-%m-%d")
selection = 'marketinsights'
directory = "./utils/data/Scraped News/"
output_file_path = f"{directory}/{selection}_data_{current_date}.csv"
df.to_csv(output_file_path, index=False)

In [ ]:
'''
Trying out sentiment scoring for article content
'''

from transformers import BertForSequenceClassification, BertTokenizer
import torch

test_content = df.iloc[5]['Article content']
print(test_content)

# initialize our model and tokenizer
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')


def get_sentiment(text):
    tokens = tokenizer.encode_plus(text, add_special_tokens=False)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    # define our starting position (0) and window size (number of tokens in each chunk)
    start = 0
    window_size = 512
    
    # initialize probabilities list
    probs_list = []
    
    start = 0
    window_size = 510  # we take 2 off here so that we can fit in our [CLS] and [SEP] tokens
    
    loop = True
    
    while loop:
        end = start + window_size
        if end >= total_len:
            loop = False
            end = total_len
        # (1) extract window from input_ids and attention_mask
        input_ids_chunk = input_ids[start:end]
        attention_mask_chunk = attention_mask[start:end]
        # (2) add [CLS] and [SEP]
        input_ids_chunk = [101] + input_ids_chunk + [102]
        attention_mask_chunk = [1] + attention_mask_chunk + [1]
        # (3) add padding upto window_size + 2 (512) tokens
        input_ids_chunk += [0] * (window_size - len(input_ids_chunk) + 2)
        attention_mask_chunk += [0] * (window_size - len(attention_mask_chunk) + 2)
        # (4) format into PyTorch tensors dictionary
        input_dict = {
            'input_ids': torch.Tensor([input_ids_chunk]).long(),
            'attention_mask': torch.Tensor([attention_mask_chunk]).int()
        }
        # (5) make logits prediction
        outputs = model(**input_dict)
        # (6) calculate softmax and append to list
        probs = torch.nn.functional.softmax(outputs[0], dim=-1)
        probs_list.append(probs)
    
        start = end

    stacks = torch.stack(probs_list)
    shape = stacks.shape
    with torch.no_grad():
        # we must include our stacks operation in here too
        stacks = torch.stack(probs_list)
        # now resize
        stacks = stacks.resize_(stacks.shape[0], stacks.shape[2])
        # finally, we can calculate the mean value for each sentiment class
        mean = stacks.mean(dim=0)
    print(mean)
    winner = torch.argmax(mean).item()
    result = ['Positive', 'Negative', 'Neutral'][winner]
    return result

df['Sentiment'] = df['Article content'].apply(get_sentiment)
for i, row in df.iterrows():
    print('Content: \n')
    print(row['Article content'])
    print('\nSentiment: \n')
    print(row['Sentiment'])
    print('**********')

In [60]:
'''
Trying out labelling missing executives from company website
'''

from serpapi import GoogleSearch
import pandas as pd
import streamlit as st
import requests
import re

from serpapi import GoogleSearch
import urllib
from bs4 import BeautifulSoup

def search_company_management(company_name, api_key):
    # Define common terms used for management page
    management_terms = {
        "Our Team", "Leadership", "Directors", 
        "Management", "Board", "Organization"
    }
    
    # Construct query with various combinations
    queries = f"({company_name} company website) AND ({' OR '.join(management_terms)})"
    
    params = {
        "q": queries,
        "engine": "google",
        "google_domain": "google.com",
        "num": 10,
        "nfpr": 1,
        "api_key": api_key,
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # Scraping each result page
    storage = []
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        }
    for temp in results['organic_results']:
        link = temp['link']
        try:
            response = requests.get(link, headers = header, timeout=10)
            response.raise_for_status()  # Raise an exception if the request was unsuccessful

            # Parse the HTML content of the page with BeautifulSoup
            html_content = urllib.parse.unquote(response.text)
            soup = BeautifulSoup(html_content, 'html.parser')
            # Extract and clean text content
            text_content = soup.get_text(separator=' ', strip=True)
            
            if text_content.strip():
                storage.append(text_content)
        except:
            print(f"Unable to scrape {link}")
    return storage, results

In [61]:
# Example usage
api_key = st.secrets['serp_api_key']
company_name = 'StubHub'
search_results, x = search_company_management(company_name, api_key)

Unable to scrape https://www.comparably.com/companies/stubhub/executive-team
Unable to scrape https://pitchbook.com/profiles/company/13230-37
Unable to scrape https://www.dnb.com/business-directory/company-profiles.stubhub_inc.30a2fb4082f9f3d30b41552d98df63ec.html


In [57]:
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Initialize BERT NER pipeline
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def extract_management_info(text, nlp):
    # Define common position titles
    position_titles = [
        "CEO", "Chief Executive Officer", 
        "CFO", "Chief Financial Officer", 
        "COO", "Chief Operating Officer", 
        "CTO", "Chief Technology Officer", 
        "CMO", "Chief Marketing Officer", 
        "CBO", "Chief Business Officer", 
        "Director", "Vice President", "Board Member",
        "VP", "President", "Chairman", "Founder", 
        "Owner", "Partner", "Head", "Independent Director"
    ]
    
    # Create a regex pattern to find names and positions
    name_pattern = r'([A-Z][a-z]+(?: [A-Z][a-z]*){0,2})'
    position_pattern = r'\b(' + '|'.join(position_titles) + r')\b'
    combined_pattern = re.compile(rf'{name_pattern}\s*,?\s*{position_pattern}|{position_pattern}\s*,?\s*{name_pattern}', re.IGNORECASE)
    
    # Find all matches in the text
    matches = combined_pattern.findall(text)
    
    # Extract names and positions
    management_info = []
    for match in matches:
        filtered_match = [m for m in match if m]
        if len(filtered_match) == 2:
            name, position = (filtered_match[0].strip(), filtered_match[1].strip())
            # Validate the name using BERT NER
            ner_results = nlp(name)
            full_name = []
            for item in ner_results:
                #print(item.keys())
                print(item)
                if item['entity_group'] == 'PER' and item['score'] >= 0.90:# and '#' not in item['word']:
                    full_name.append(item['word'])
            if full_name:
                management_info.append((' '.join(full_name), position))
    return management_info

def clean_string(input_string):
    # Use regex to replace non-alphanumeric characters with an empty string
    cleaned_string = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return cleaned_string

def find_full_word(text, substring):
    # Find the first occurrence of the case-sensitive substring in the text using regex
    pattern = r'\b\w*' + re.escape(substring) + r'\w*\b'
    match = re.search(pattern, text)
    
    if match:
        # Return the full word containing the substring
        return match.group(0)
    else:
        return None

# Example usage
text = "This is an example sentence. Let's find the Substring in this text."
substring = "Sub"
result = find_full_word(text, substring)
print(result)  # Output: Substring

name_position_pairs = []
for key in search_results:
    text = clean_string(key)
    #print(text)
    management_info = extract_management_info(text, nlp)
    #print(management_info)
    name_position_pairs.extend(management_info)
#print(name_position_pairs)
# Count occurrences of each name-position pair
pair_counter = Counter(name_position_pairs)

# Find the most common pairs
most_common_pairs = pair_counter.most_common()

# print the most common name-position pairs
for pair, count in most_common_pairs:
    print(f"Name: {pair[0]}, Position: {pair[1]}, Count: {count}")



/Users/launchpad/miniforge3/envs/argus/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity_group': 'PER', 'score': 0.9991996, 'word': 'Art Yegorov', 'start': 9, 'end': 20}
{'entity_group': 'PER', 'score': 0.9996076, 'word': 'C', 'start': 0, 'end': 1}
{'entity_group': 'PER', 'score': 0.9953915, 'word': '##risto', 'start': 1, 'end': 6}
{'entity_group': 'PER', 'score': 0.7530372, 'word': '##pher Miller', 'start': 6, 'end': 17}
{'entity_group': 'PER', 'score': 0.9991628, 'word': 'Na', 'start': 4, 'end': 6}
{'entity_group': 'PER', 'score': 0.9269295, 'word': '##ku', 'start': 6, 'end': 8}
{'entity_group': 'PER', 'score': 0.9974962, 'word': '##l Paul', 'start': 8, 'end': 14}
{'entity_group': 'PER', 'score': 0.9994564, 'word': 'Julie Jacob', 'start': 5, 'end': 16}
{'entity_group': 'PER', 'score': 0.98797685, 'word': 'Na', 'start': 5, 'end': 7}
{'entity_group': 'PER', 'score': 0.94521886, 'word': '##l Paul', 'start': 9, 'end': 15}
{'entity_group': 'PER', 'score': 0.8243458, 'word': 'Yegorov', 'start': 9, 'end': 16}
{'entity_group': 'ORG', 'score': 0.7328082, 'word': 'Megan R